**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

### Raphaël Ginoulhac

In [0]:
#Only execute this on google colab
!pip install scikit-video

     |████████████████████████████████| 2.3MB 26.0MB/s 


In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

With probability $\epsilon$, the agent chooses an action at random, and with probability $1-\epsilon$, the agent picks the action that maximizes the Q-function for the current state (i.e., the best action according to its knowledge so far).  
Epsilon is essential as it allows the agent to explore its environment. Indeed, as in all problems in Reinforcement Learning, there is a tradeoff to be made between exploration and exploitation. The agent has to learn the optimal Q-function by acquiring knowledge, for example by taking random actions. If the agent only acted greedily, it would be stuck in a suboptimal strategy depending on the initialization of the Q-function : it would never visit some states or state-action pairs, and thus couldn't form a correct estimate of their Q-values.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=51 # set small when debugging
epochs_test=51 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The grid is extended with 2 unattainable squares on every side, so that the agent can still see 2 squares ahead even if it is on a side of the actual grid.  
The board array stores the positions of the bonus and malus squares (and their values, 0.5 for a bonus and -1 for a malus, 0 elsewhere).  
The position array contains a 1 where the player is, -1 on the extra squares added on the side where the agent cannot move to, and 0 elsewhere.  
Both arrays make up the state that the agent has access to, with a visibility of 2 squares ahead.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        #take an action at random
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False
    
        win = 0
        lose = 0
    
        while not game_over:
            # The agent performs the greedy action according to Q (no exploration in test phase)
            action = agent.act(state, train=False)
    
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
    
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        #no reinforce, as it is a test
            
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
        
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent_random = RandomAgent()

test(agent_random,env,epochs_test,prefix='random')
HTML(display_videos('random50.mp4'))

Win/lose count 6.5/14.0. Average score (-7.5)
Win/lose count 9.5/17.0. Average score (-7.5)
Win/lose count 9.5/17.0. Average score (-7.5)
Win/lose count 11.0/12.0. Average score (-5.875)
Win/lose count 9.0/18.0. Average score (-6.5)
Win/lose count 10.5/20.0. Average score (-7.0)
Win/lose count 5.5/15.0. Average score (-7.357142857142857)
Win/lose count 10.5/14.0. Average score (-6.875)
Win/lose count 9.5/20.0. Average score (-7.277777777777778)
Win/lose count 10.5/7.0. Average score (-6.2)
Win/lose count 6.5/9.0. Average score (-5.863636363636363)
Win/lose count 5.0/6.0. Average score (-5.458333333333333)
Win/lose count 12.5/23.0. Average score (-5.846153846153846)
Win/lose count 7.0/14.0. Average score (-5.928571428571429)
Win/lose count 6.5/5.0. Average score (-5.433333333333334)
Win/lose count 12.0/18.0. Average score (-5.46875)
Win/lose count 11.5/17.0. Average score (-5.470588235294118)
Win/lose count 3.5/3.0. Average score (-5.138888888888889)
Win/lose count 9.5/13.0. Average sco

The random agent may act as a (poor) baseline, it gets around -4.9 points on average

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p^\pi(.,.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim p(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim p(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




The definition of the Q-function is : 
\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}
We can extract the first term of the sum :
$Q^{\pi}(s,a) = \mathbb{E}_{p^\pi} \Big[ r(s,a) + \gamma \sum_{t\leq T}\gamma^{t}r(s_{t+1},a_{t+1}) | s_0 = s, a_0 = a \Big]$

We have, by reindexing the time : $\mathbb{E}_{p^\pi} \Big[\sum_{t\leq T}\gamma^{t}r(s_{t+1},a_{t+1}) | s_0 = s, a_0 = a \Big] = E_{(s',a')\sim p(.,.|s,a)}\Big[\mathbb{E}_{p^\pi} \Big[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t}) | s_0 = s', a_0 = a' \Big]\Big] =  E_{(s',a')\sim p(.,.|s,a)}[ Q^{\pi}(s',a') ]$

Thus by linearity of the expectation (and as $r(s,a)$ is independent of $p$ and $\pi$ it is equal to its expectation regard to any of those two) :  \begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.,.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}
_____________________________________

The optimal Q-function is :  
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) 
= \max_{\pi}E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] 
= \max_{\pi}\mathbb{E}_{p^\pi} \Big[ r(s,a) + \gamma \sum_{t\leq T}\gamma^{t}r(s_{t+1},a_{t+1}) | s_0 = s, a_0 = a \Big]
= r(s,a) + \gamma \max_{\pi}\mathbb{E}_{p^\pi}\Big[\sum_{t\leq T}\gamma^{t}r(s_{t+1},a_{t+1}) | s_0 = s, a_0 = a \Big] 
= r(s,a) + \gamma \max_{\pi}E_{(s',a')\sim p(.,.|s,a)}\Big[\mathbb{E}_{p^\pi} \Big[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t}) | s_0 = s', a_0 = a' \Big]\Big]
=r(s,a) + \gamma \max_{\pi}E_{(s',a')\sim p(.,.|s,a)}[ Q^{\pi}(s',a') ]
=E_{s'\sim p(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].\> .
\end{equation*}
as maximizing over all possible $\pi$ is equivalent to optimizing over the best action $a'$ when in the next state $s'$ given by the optimal policy, then following the optimal policy.
_____________________

$Q^*$ is the only fixed point of the optimal Bellman equation, and by theorem, applying it repeatedly on any Q will make it converge to $Q^*$.

Minimizing this objective will ensure that the Q-function parametrized by $\theta$ gets close to the optimal Q-function. We use the square loss to get a differentiable loss.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()
        #index where we will store the next transition in memory
        self.pointer = 0

    def remember(self, m):
        if len(self.memory)<self.max_memory:
          self.memory.append(m)
        else:
          #if the memory is full, we replace the oldest observation with the newest one
          self.memory[self.pointer] = m
        self.pointer = (self.pointer + 1) % self.max_memory

    def random_access(self):
        return self.memory[np.random.randint(0, len(self.memory), size=1)[0]]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch-1, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        #Take the argmax of the Q-function
        return np.argmax(self.model.predict(np.expand_dims(s, axis=0))[0])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            #retrieve a transition
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_
            
            #set the target Q, it is y_j in the paper
            #we do not change the Q value for the actions we did not take
            target_q[i] = self.model.predict(np.expand_dims(s_, axis=0))[0]
            #Q target for the action that was taken
            if game_over_:
                target_q[i][a_] = r_
            else:
                target_q[i][a_] = r_ + self.discount * np.max(self.model.predict(np.expand_dims(n_s_, axis=0))[0])

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("adam", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model, 2 fully connected layers with dimensions 32 and 4 (number of actions)
        
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state)))
        model.add(Dense(32))
        model.add(Activation('relu')) #non-linearity
        #model.add(Dense(32))
       # model.add(Activation('relu'))
        model.add(Dense(4))
        model.add(Activation('softmax')) #to get a probability distribution on the actions
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
       # model.compile(adam(lr=lr), "mse")
        self.model = model
        

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent_fc, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train50.mp4'))

Epoch 000/050 | Loss 0.0083 | Win/lose count 3.0/6.0 (-3.0)
Epoch 001/050 | Loss 0.0028 | Win/lose count 3.0/8.0 (-5.0)
Epoch 002/050 | Loss 0.0012 | Win/lose count 1.0/3.0 (-2.0)
Epoch 003/050 | Loss 0.0095 | Win/lose count 3.0/7.0 (-4.0)
Epoch 004/050 | Loss 0.0238 | Win/lose count 3.5/4.0 (-0.5)
Epoch 005/050 | Loss 0.0115 | Win/lose count 2.0/4.0 (-2.0)
Epoch 006/050 | Loss 0.0081 | Win/lose count 5.0/7.0 (-2.0)
Epoch 007/050 | Loss 0.0102 | Win/lose count 4.0/7.0 (-3.0)
Epoch 008/050 | Loss 0.0157 | Win/lose count 6.0/7.0 (-1.0)
Epoch 009/050 | Loss 0.0086 | Win/lose count 1.5/6.0 (-4.5)
Epoch 010/050 | Loss 0.0180 | Win/lose count 5.0/8.0 (-3.0)
Epoch 011/050 | Loss 0.0165 | Win/lose count 4.0/6.0 (-2.0)
Epoch 012/050 | Loss 0.0147 | Win/lose count 4.5/4.0 (0.5)
Epoch 013/050 | Loss 0.0079 | Win/lose count 3.0/3.0 (0.0)
Epoch 014/050 | Loss 0.0006 | Win/lose count 4.0/3.0 (1.0)
Epoch 015/050 | Loss 0.0087 | Win/lose count 2.0/5.0 (-3.0)
Epoch 016/050 | Loss 0.0163 | Win/lose coun

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        #2 2D Convolutions of size 64 and 32 with a kernel size of (2,2), then we flatten the hidden state and apply a dense layer
        #Maxpooling, dropout, batchnorm possible
        model = Sequential()
        model.add(Conv2D(64, (2,2), input_shape=(5,5,self.n_state)))
        model.add(Activation('relu'))
        model.add(Conv2D(32, (2,2)))
        model.add(Activation('relu'))
        model.add(Flatten())
        model.add(Dense(4))
        model.add(Activation('softmax'))
        
        #model.compile(adam(lr=lr), "mse")
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")

        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent_cnn,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train50.mp4'))

Epoch 000/050 | Loss 0.0094 | Win/lose count 2.5/5.0 (-2.5)
Epoch 001/050 | Loss 0.0000 | Win/lose count 6.0/3.0 (3.0)
Epoch 002/050 | Loss 0.0040 | Win/lose count 5.5/4.0 (1.5)
Epoch 003/050 | Loss 0.0020 | Win/lose count 5.5/6.0 (-0.5)
Epoch 004/050 | Loss 0.0099 | Win/lose count 3.5/1.0 (2.5)
Epoch 005/050 | Loss 0.0002 | Win/lose count 3.5/0 (3.5)
Epoch 006/050 | Loss 0.0116 | Win/lose count 2.0/2.0 (0.0)
Epoch 007/050 | Loss 0.0124 | Win/lose count 1.0/2.0 (-1.0)
Epoch 008/050 | Loss 0.0082 | Win/lose count 1.0/3.0 (-2.0)
Epoch 009/050 | Loss 0.0040 | Win/lose count 3.5/1.0 (2.5)
Epoch 010/050 | Loss 0.0041 | Win/lose count 4.5/3.0 (1.5)
Epoch 011/050 | Loss 0.0198 | Win/lose count 3.5/3.0 (0.5)
Epoch 012/050 | Loss 0.0064 | Win/lose count 2.5/4.0 (-1.5)
Epoch 013/050 | Loss 0.0029 | Win/lose count 1.0/2.0 (-1.0)
Epoch 014/050 | Loss 0.0024 | Win/lose count 1.5/2.0 (-0.5)
Epoch 015/050 | Loss 0.0000 | Win/lose count 3.5/3.0 (0.5)
Epoch 016/050 | Loss 0.0038 | Win/lose count 1.5/4.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.05, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.05, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 1.0/0. Average score (1.0)
Win/lose count 0/0. Average score (0.5)
Win/lose count 0/0. Average score (0.3333333333333333)
Win/lose count 1.0/0. Average score (0.5)
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 1.0/0. Average score (0.5833333333333334)
Win/lose count 0/0. Average score (0.5)
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 1.0/0. Average score (0.55)
Win/lose count 0/0. Average score (0.5)
Win/lose count 1.5/0. Average score (0.5833333333333334)
Win/lose count 1.5/0. Average score (0.6538461538461539)
Win/lose count 0.5/0. Average score (0.6428571428571429)
Win/lose count 0.5/0. Average score (0.6333333333333333)
Win/lose count 1.5/0. Average score (0.6875)
Win/lose count 0/0. Average score (0.6470588235294118)
Win/lose count 2.0/0. Average score (0.7222222222222222)
Win/lose count 1.0/0. Average score (0.7368421052631579)
Win/lose count 0/0. Average score (0.7)
Win/lose count 2.

The CNN gets an average score of 0.7 while the FC gets 1.2. It is much better than the random baseline, and is positive which proves the agents are learning.
We get to this performance in 50 epochs without much hyperparameter or architecture tuning.
Important : I first used a more complex model with 3 layers of 64, 64 and 4 neurons for the FC, and the adam optimizer, but it did not yield better results.
The videos often show the agent alternating between the same 2 squares : it does not explore the board enough even when there are visible bonuses.

In [0]:
print("CNN Test")
HTML(display_videos('cnn_test50.mp4'))

CNN Test


In [0]:
print("FC Test")
HTML(display_videos('fc_test50.mp4'))

FC Test


#### Changing the temperature

In [0]:
temperatures = [0.1, 0.5, 0.8]
for temp in temperatures:
    print("Temperature : ", temp)
    env = Environment(grid_size=size, max_time=T,temperature=temp)
    agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

    agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
    print('Test of the CNN')
    test(agent_cnn,env,epochs_test,prefix='cnn_test')
    print('Test of the FC')
    test(agent_fc,env,epochs_test,prefix='fc_test')
    print("\n")

Temperature :  0.1
Test of the CNN
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 1.0/0. Average score (0.75)
Win/lose count 1.0/0. Average score (0.8333333333333334)
Win/lose count 1.0/0. Average score (0.875)
Win/lose count 0/0. Average score (0.7)
Win/lose count 0.5/0. Average score (0.6666666666666666)
Win/lose count 0.5/0. Average score (0.6428571428571429)
Win/lose count 0/0. Average score (0.5625)
Win/lose count 0/0. Average score (0.5)
Win/lose count 0/0. Average score (0.45)
Win/lose count 0/0. Average score (0.4090909090909091)
Win/lose count 0/0. Average score (0.375)
Win/lose count 1.0/0. Average score (0.4230769230769231)
Win/lose count 1.0/0. Average score (0.4642857142857143)
Win/lose count 1.0/0. Average score (0.5)
Win/lose count 1.5/0. Average score (0.5625)
Win/lose count 0/0. Average score (0.5294117647058824)
Win/lose count 1.0/0. Average score (0.5555555555555556)
Win/lose count 0/0. Average score (0.5263157894736842)
Win/lose count 0/0. Average score (0

In [0]:
print("CNN Test with high temperature")
HTML(display_videos('cnn_test50.mp4'))

CNN Test with high temperature


In [0]:
print("FC Test with high temperature")
HTML(display_videos('fc_test50.mp4'))

FC Test with high temperature


The performance increases with the temperature as the number of bonus squares also increases. We still observe the same "alternating" behavior which needs to be fixed.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    eps=1

    for e in range(epoch):
        #We linearly decrease epsilon from 1 to 0.1 at each epoch
        eps -= (1-0.1)/epoch
        agent.set_epsilon(eps)

        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True) if isinstance(env, EnvironmentExploring) else env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch-1, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1, malus_position_=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        self.malus_position_ = malus_position_ # malus if we visit a position more than once

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        #malus given to every square that we have already visited
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = self.malus_position_ 

        reward = reward + self.board[self.x, self.y]

        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

### Training with a better exploration policy (no EnvironmentExploring yet)

In [0]:
# Training with a better exploration policy for the CNN
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent_cnn_exp = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=2)
train_explore(agent_cnn_exp, env, epoch=epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore50.mp4'))

Epoch 000/050 | Loss 0.0096 | Win/lose count 9.0/9.0 (0.0)
Epoch 001/050 | Loss 0.0254 | Win/lose count 9.0/18.0 (-9.0)
Epoch 002/050 | Loss 0.0221 | Win/lose count 7.5/18.0 (-10.5)
Epoch 003/050 | Loss 0.0187 | Win/lose count 11.5/16.0 (-4.5)
Epoch 004/050 | Loss 0.0237 | Win/lose count 8.5/13.0 (-4.5)
Epoch 005/050 | Loss 0.0179 | Win/lose count 8.0/18.0 (-10.0)
Epoch 006/050 | Loss 0.0367 | Win/lose count 9.0/6.0 (3.0)
Epoch 007/050 | Loss 0.0158 | Win/lose count 10.5/8.0 (2.5)
Epoch 008/050 | Loss 0.0221 | Win/lose count 15.5/10.0 (5.5)
Epoch 009/050 | Loss 0.0249 | Win/lose count 8.0/19.0 (-11.0)
Epoch 010/050 | Loss 0.0172 | Win/lose count 11.5/10.0 (1.5)
Epoch 011/050 | Loss 0.0177 | Win/lose count 8.0/16.0 (-8.0)
Epoch 012/050 | Loss 0.0104 | Win/lose count 10.5/19.0 (-8.5)
Epoch 013/050 | Loss 0.0099 | Win/lose count 9.5/9.0 (0.5)
Epoch 014/050 | Loss 0.0158 | Win/lose count 16.0/11.0 (5.0)
Epoch 015/050 | Loss 0.0180 | Win/lose count 14.5/9.0 (5.5)
Epoch 016/050 | Loss 0.0190

In [0]:
# Evaluation
test(agent_cnn_exp,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore50.mp4'))

Win/lose count 1.0/0. Average score (1.0)
Win/lose count 6.0/0. Average score (3.5)
Win/lose count 3.0/0. Average score (3.3333333333333335)
Win/lose count 2.0/0. Average score (3.0)
Win/lose count 2.0/0. Average score (2.8)
Win/lose count 2.0/0. Average score (2.6666666666666665)
Win/lose count 2.0/0. Average score (2.5714285714285716)
Win/lose count 6.0/0. Average score (3.0)
Win/lose count 5.0/0. Average score (3.2222222222222223)
Win/lose count 7.5/0. Average score (3.65)
Win/lose count 3.5/0. Average score (3.6363636363636362)
Win/lose count 4.5/0. Average score (3.7083333333333335)
Win/lose count 4.0/0. Average score (3.730769230769231)
Win/lose count 2.5/0. Average score (3.642857142857143)
Win/lose count 2.0/0. Average score (3.533333333333333)
Win/lose count 2.0/0. Average score (3.4375)
Win/lose count 4.5/0. Average score (3.5)
Win/lose count 4.5/0. Average score (3.5555555555555554)
Win/lose count 4.0/0. Average score (3.5789473684210527)
Win/lose count 4.0/0. Average score 

In [0]:
# Training with a better exploration policy for the FC
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent_fc_exp = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=2)
train_explore(agent_fc_exp, env, epoch=epochs_train, prefix='fc_train_explore')
HTML(display_videos('fc_train_explore50.mp4'))

Epoch 000/050 | Loss 0.0135 | Win/lose count 9.0/2.0 (7.0)
Epoch 001/050 | Loss 0.0206 | Win/lose count 6.0/13.0 (-7.0)
Epoch 002/050 | Loss 0.0151 | Win/lose count 7.0/11.0 (-4.0)
Epoch 003/050 | Loss 0.0059 | Win/lose count 8.5/4.0 (4.5)
Epoch 004/050 | Loss 0.0149 | Win/lose count 9.0/4.0 (5.0)
Epoch 005/050 | Loss 0.0266 | Win/lose count 8.0/14.0 (-6.0)
Epoch 006/050 | Loss 0.0146 | Win/lose count 14.0/14.0 (0.0)
Epoch 007/050 | Loss 0.0158 | Win/lose count 8.5/14.0 (-5.5)
Epoch 008/050 | Loss 0.0236 | Win/lose count 13.0/17.0 (-4.0)
Epoch 009/050 | Loss 0.0270 | Win/lose count 3.5/7.0 (-3.5)
Epoch 010/050 | Loss 0.0205 | Win/lose count 10.5/16.0 (-5.5)
Epoch 011/050 | Loss 0.0286 | Win/lose count 9.5/14.0 (-4.5)
Epoch 012/050 | Loss 0.0183 | Win/lose count 6.5/4.0 (2.5)
Epoch 013/050 | Loss 0.0232 | Win/lose count 13.0/9.0 (4.0)
Epoch 014/050 | Loss 0.0235 | Win/lose count 12.5/11.0 (1.5)
Epoch 015/050 | Loss 0.0165 | Win/lose count 9.5/12.0 (-2.5)
Epoch 016/050 | Loss 0.0206 | Wi

In [0]:
# Evaluation
test(agent_fc_exp,env,epochs_test,prefix='fc_test_explore')
HTML(display_videos('fc_test_explore50.mp4'))

Win/lose count 3.0/1.0. Average score (2.0)
Win/lose count 1.0/0. Average score (1.5)
Win/lose count 0/0. Average score (1.0)
Win/lose count 5.5/0. Average score (2.125)
Win/lose count 3.5/0. Average score (2.4)
Win/lose count 3.5/0. Average score (2.5833333333333335)
Win/lose count 3.5/2.0. Average score (2.4285714285714284)
Win/lose count 2.0/0. Average score (2.375)
Win/lose count 4.5/0. Average score (2.611111111111111)
Win/lose count 3.5/1.0. Average score (2.6)
Win/lose count 3.0/0. Average score (2.6363636363636362)
Win/lose count 2.5/0. Average score (2.625)
Win/lose count 1.0/0. Average score (2.5)
Win/lose count 1.0/0. Average score (2.392857142857143)
Win/lose count 0.5/0. Average score (2.2666666666666666)
Win/lose count 5.0/0. Average score (2.4375)
Win/lose count 3.5/0. Average score (2.5)
Win/lose count 2.5/0. Average score (2.5)
Win/lose count 0/0. Average score (2.3684210526315788)
Win/lose count 0.5/0. Average score (2.275)
Win/lose count 3.5/0. Average score (2.33333

We get 3 points on average for the CNN and for the FC which is a significant improvement. The performance of the agent is poor during learning because it explores a lot, but it enables it to learn about its environment (epsilon varies from 1 to 0.1). During test time, it only acts greedily and does not explore.

### Training with a better exploration policy and EnvironmentExploring

In [0]:
# Training with a better exploration policy for the CNN and EnvironmentExploring
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3, malus_position_= 0.2)
agent_cnn_exp2 = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=3000, batch_size = 16,n_state=3)
train_explore(agent_cnn_exp2, env, epoch=epochs_train, prefix='cnn_train_explore2_')
HTML(display_videos('cnn_train_explore2_50.mp4'))

Epoch 000/050 | Loss 0.0668 | Win/lose count 11.0/42.200000000000024 (-31.200000000000024)
Epoch 001/050 | Loss 0.0343 | Win/lose count 8.5/39.60000000000002 (-31.100000000000023)
Epoch 002/050 | Loss 0.0063 | Win/lose count 11.5/43.20000000000004 (-31.70000000000004)
Epoch 003/050 | Loss 0.0166 | Win/lose count 7.5/35.6 (-28.1)
Epoch 004/050 | Loss 0.0167 | Win/lose count 5.0/41.00000000000006 (-36.00000000000006)
Epoch 005/050 | Loss 0.0380 | Win/lose count 11.5/39.400000000000006 (-27.900000000000006)
Epoch 006/050 | Loss 0.0293 | Win/lose count 4.5/42.20000000000002 (-37.70000000000002)
Epoch 007/050 | Loss 0.0205 | Win/lose count 11.5/37.19999999999998 (-25.69999999999998)
Epoch 008/050 | Loss 0.0246 | Win/lose count 11.0/39.60000000000003 (-28.60000000000003)
Epoch 009/050 | Loss 0.0268 | Win/lose count 15.5/38.40000000000003 (-22.900000000000027)
Epoch 010/050 | Loss 0.0294 | Win/lose count 6.5/37.199999999999996 (-30.699999999999996)
Epoch 011/050 | Loss 0.0113 | Win/lose count

In [0]:
# Evaluation
test(agent_cnn_exp2,env,epochs_test,prefix='cnn_test_explore2_')
HTML(display_videos('cnn_test_explore2_50.mp4'))

Win/lose count 1.0/0. Average score (1.0)
Win/lose count 8.0/0. Average score (4.5)
Win/lose count 5.0/0. Average score (4.666666666666667)
Win/lose count 18.5/0. Average score (8.125)
Win/lose count 3.0/0. Average score (7.1)
Win/lose count 6.0/0. Average score (6.916666666666667)
Win/lose count 11.0/0. Average score (7.5)
Win/lose count 5.0/0. Average score (7.1875)
Win/lose count 10.5/0. Average score (7.555555555555555)
Win/lose count 11.5/1.0. Average score (7.85)
Win/lose count 10.5/3.0. Average score (7.818181818181818)
Win/lose count 7.0/0. Average score (7.75)
Win/lose count 15.0/0. Average score (8.307692307692308)
Win/lose count 10.5/0. Average score (8.464285714285714)
Win/lose count 8.0/0. Average score (8.433333333333334)
Win/lose count 7.0/0. Average score (8.34375)
Win/lose count 11.0/0. Average score (8.5)
Win/lose count 4.5/0. Average score (8.277777777777779)
Win/lose count 4.5/0. Average score (8.078947368421053)
Win/lose count 8.5/1.0. Average score (8.05)
Win/lose

In [0]:
# Training with a better exploration policy for the FC and EnvironmentExploring
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3, malus_position_= 0.3)
agent_fc_exp2 = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=3000, batch_size = 16, n_state=3)
train_explore(agent_fc_exp2, env, epoch=epochs_train, prefix='fc_train_explore2_')
HTML(display_videos('fc_train_explore2_50.mp4'))

Epoch 000/050 | Loss 0.0158 | Win/lose count 8.0/48.3999999999999 (-40.3999999999999)
Epoch 001/050 | Loss 0.0225 | Win/lose count 9.0/56.59999999999983 (-47.59999999999983)
Epoch 002/050 | Loss 0.0282 | Win/lose count 10.5/48.39999999999989 (-37.89999999999989)
Epoch 003/050 | Loss 0.0142 | Win/lose count 6.0/53.49999999999986 (-47.49999999999986)
Epoch 004/050 | Loss 0.0474 | Win/lose count 9.0/54.49999999999991 (-45.49999999999991)
Epoch 005/050 | Loss 0.0418 | Win/lose count 12.0/52.19999999999987 (-40.19999999999987)
Epoch 006/050 | Loss 0.0275 | Win/lose count 10.0/53.79999999999986 (-43.79999999999986)
Epoch 007/050 | Loss 0.0198 | Win/lose count 14.0/50.49999999999987 (-36.49999999999987)
Epoch 008/050 | Loss 0.0333 | Win/lose count 13.0/51.79999999999992 (-38.79999999999992)
Epoch 009/050 | Loss 0.0385 | Win/lose count 10.5/55.5999999999999 (-45.0999999999999)
Epoch 010/050 | Loss 0.0245 | Win/lose count 14.5/47.39999999999993 (-32.89999999999993)
Epoch 011/050 | Loss 0.0179 |

In [0]:
# Evaluation
test(agent_fc_exp2,env,epochs_test,prefix='fc_test_explore2_')
HTML(display_videos('fc_test_explore2_50.mp4'))

Win/lose count 9.5/0. Average score (9.5)
Win/lose count 7.5/1.0. Average score (8.0)
Win/lose count 13.5/1.0. Average score (9.5)
Win/lose count 2.0/0. Average score (7.625)
Win/lose count 4.0/0. Average score (6.9)
Win/lose count 5.0/0. Average score (6.583333333333333)
Win/lose count 1.5/0. Average score (5.857142857142857)
Win/lose count 10.5/0. Average score (6.4375)
Win/lose count 15.5/0. Average score (7.444444444444445)
Win/lose count 4.5/0. Average score (7.15)
Win/lose count 0/0. Average score (6.5)
Win/lose count 18.0/1.0. Average score (7.375)
Win/lose count 10.5/1.0. Average score (7.538461538461538)
Win/lose count 4.0/0. Average score (7.285714285714286)
Win/lose count 9.5/1.0. Average score (7.366666666666666)
Win/lose count 12.0/0. Average score (7.65625)
Win/lose count 1.5/0. Average score (7.294117647058823)
Win/lose count 14.0/0. Average score (7.666666666666667)
Win/lose count 3.5/0. Average score (7.447368421052632)
Win/lose count 4.0/0. Average score (7.275)
Win/l

The scores we obtain after using the decaying epsilon technique and the environment that encourages exploring are 8.8 for the CNN and around 8.2 for the FC agent. 
The hyperparameters used were : a learning rate of 0.1 with sgd, a memory size of 3000, a batch size of 16 and a malus on EnvironmentExploring of 0.2 compared to the 0.1 suggested.  
During training, this worsens the score of the agent, but it learns that it needs to explore the board, and the results prove that it is successful.
We should encourage the agent to explore even more, out of its sight, so that it does not oscillate between two positions if the only bonuses left are on the other side of the board.
We can still improve on the architecture of the two networks and the hyperparameters to get the mean score up to 10 or 12.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [0]:
class DQN_mimick(DQN):
    def __init__(self, *args, lr=0.1, **kwargs):
        super(DQN_mimick, self).__init__( *args,**kwargs)
        
        
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state)))
        model.add(Dense(64))
        model.add(Activation('relu')) #non-linearity
        #model.add(Dense(64))
        #model.add(Activation('relu'))

        model.add(Dense(4))
        model.add(Activation('softmax')) #to get a probability distribution on the actions
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        #model.compile(adam(lr=lr), "mse")
        self.model = model


    def reinforce(self, s_, n_s_, a_, r_, game_over_):
    # Two steps: first memorize the states, second learn from the pool

      self.memory.remember([s_, n_s_, a_, r_, game_over_])
      
      input_states = np.zeros((self.batch_size, 5,5,self.n_state))
      target_q = np.zeros((self.batch_size, 4))
      
      for i in range(self.batch_size):
          #retrieve a transition
          s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
          input_states[i] = s_
          
          #set the target Q as the Q-value from the dqn network we're copying

          #We provoke a strong update in the direction of the action that the expert has taken, while giving a 0 expected reward to the other actions
          target_q[i][a_] = 3
      # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
      #target_q = np.clip(target_q, -3, 3)

      l = self.model.train_on_batch(input_states, target_q)

      return l

In [0]:
def train_mimick(expert, agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = expert.act(state, False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch-1, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [0]:
# Training with a better exploration policy for the FC and EnvironmentExploring
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3, malus_position_= 0.2)
agent_copy = DQN_mimick(size, lr=.1, epsilon = 0.1, memory_size=3000, batch_size = 16, n_state=3)
train_mimick(agent_fc_exp2, agent_copy, env, epoch=epochs_train, prefix='fc_copy')
HTML(display_videos('fc_copy50.mp4'))

Epoch 000/050 | Loss 1.0219 | Win/lose count 2.0/1.0 (1.0)
Epoch 001/050 | Loss 1.0983 | Win/lose count 10.0/0 (10.0)
Epoch 002/050 | Loss 1.1803 | Win/lose count 12.5/0 (12.5)
Epoch 003/050 | Loss 1.2035 | Win/lose count 8.0/0 (8.0)
Epoch 004/050 | Loss 1.3062 | Win/lose count 4.5/0 (4.5)
Epoch 005/050 | Loss 1.1336 | Win/lose count 17.5/0 (17.5)
Epoch 006/050 | Loss 1.1414 | Win/lose count 14.0/1.0 (13.0)
Epoch 007/050 | Loss 1.0426 | Win/lose count 0.5/0 (0.5)
Epoch 008/050 | Loss 1.1170 | Win/lose count 10.5/0 (10.5)
Epoch 009/050 | Loss 1.0428 | Win/lose count 4.0/0 (4.0)
Epoch 010/050 | Loss 1.2188 | Win/lose count 13.5/0 (13.5)
Epoch 011/050 | Loss 1.2875 | Win/lose count 5.0/0 (5.0)
Epoch 012/050 | Loss 1.1328 | Win/lose count 4.5/0 (4.5)
Epoch 013/050 | Loss 1.0294 | Win/lose count 2.0/0 (2.0)
Epoch 014/050 | Loss 1.1349 | Win/lose count 5.5/0 (5.5)
Epoch 015/050 | Loss 1.0156 | Win/lose count 1.0/0 (1.0)
Epoch 016/050 | Loss 1.1298 | Win/lose count 1.5/0 (1.5)
Epoch 017/050 |

In [0]:
# Evaluation
test(agent_copy,env,epochs_test,prefix='fc_copy')
HTML(display_videos('fc_copy50.mp4'))

Win/lose count 1.0/0. Average score (1.0)
Win/lose count 3.0/0. Average score (2.0)
Win/lose count 3.5/0. Average score (2.5)
Win/lose count 7.0/0. Average score (3.625)
Win/lose count 3.5/0. Average score (3.6)
Win/lose count 4.0/0. Average score (3.6666666666666665)
Win/lose count 2.5/0. Average score (3.5)
Win/lose count 5.0/0. Average score (3.6875)
Win/lose count 3.5/0. Average score (3.6666666666666665)
Win/lose count 6.0/0. Average score (3.9)
Win/lose count 2.5/0. Average score (3.772727272727273)
Win/lose count 7.0/0. Average score (4.041666666666667)
Win/lose count 5.0/0. Average score (4.115384615384615)
Win/lose count 7.0/1.0. Average score (4.25)
Win/lose count 3.0/0. Average score (4.166666666666667)
Win/lose count 5.0/0. Average score (4.21875)
Win/lose count 4.0/0. Average score (4.205882352941177)
Win/lose count 7.0/0. Average score (4.361111111111111)
Win/lose count 0.5/0. Average score (4.157894736842105)
Win/lose count 1.5/0. Average score (4.025)
Win/lose count 5.0

This mimicking network gets a 3.74 score which is fairly good. It shows that the expert has learned a good policy that can generalize to most state/action pairs, and thus that copying it is a good idea to learn quickly and without exploration.

***